In [2]:
# !sudo pip install --upgrade transformers
# !sudo pip install fuzzywuzzy

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
import pandas as pd
import transformers
import time
from tqdm import tqdm
from fuzzywuzzy import process
print(transformers.__version__)

4.35.0


/opt/conda/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
model_name ="openchat/openchat_3.5"
# model_mistral = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
#     device_map=device_map)
model_open_chat = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16,offload_folder="offload")
tokenizer_open_chat = AutoTokenizer.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [1]:
import torch
from transformers import pipeline

#pipe = pipeline("text-generation", model="HuggingFaceH4/starchat-beta", torch_dtype=torch.bfloat16, device_map="auto")
pipe = pipeline("text-generation", model="/packages/huggingface/cache/models--HuggingFaceH4--starchat-beta/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f/", torch_dtype=torch.bfloat16, device_map="cpu")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of GPTBigCodeForCausalLM were not initialized from the model checkpoint at /packages/huggingface/cache/models--HuggingFaceH4--starchat-beta/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f/ and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [10]:
import time
from datetime import datetime
print("Today's date:", datetime.now())
t1 = time.time()
print(t1)
prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
prompt = prompt_template.format(query="How do I sort a list in Python?")
print("before model start")
# We use a special <|end|> token with ID 49155 to denote ends of a turn
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)
print("aavesh")
t2 = time.time()
t2-t1
print("Today's date:", datetime.now())
print("ended the pipe")
print(outputs)

Setting `pad_token_id` to `eos_token_id`:49155 for open-end generation.


Today's date: 2023-11-09 11:58:02.727955
1699531082.7280452
before model start
aavesh
Today's date: 2023-11-09 11:59:41.961743
ended the pipe
[{'generated_text': "<|system|>\n<|end|>\n<|user|>\nHow do I sort a list in Python?<|end|>\n<|assistant|>\nThere are multiple ways to sort a list in Python. One of the most common ways is to use the sort() method.\n\nHere's an example:\n\n```\nnumbers = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]\nnumbers.sort()\nprint(numbers)\n```\n\nThis will sort the list in place and print the sorted list.\n\nAnother way to sort a list is to use the sorted() function, which returns a new sorted list without modifying the original list.\n\nHere's an example:\n\n```\nnumbers = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]\nsorted_numbers = sorted(numbers)\nprint(sorted_numbers)\n```\n\nBoth methods support various sorting algorithms and can sort a list in either ascending or descending order."}]


In [ ]:
# We use a variant of ChatML to format each message
import time

t1 = time.time()

prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
prompt = prompt_template.format(query="How do I sort a list in Python?")
print("before model start")
# We use a special <|end|> token with ID 49155 to denote ends of a turn
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)
print("aavesh")
t2 = time.time()
t2-t1

In [ ]:
outputs[0]["generated_text"].split("\n")

In [ ]:
#!pip install transformers==4.34.1

In [3]:
from refractml import *
from refractml.constants import MLModelFlavours

# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [4]:
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self,model_name):
        super().__init__()

        self.model_loaded = None
        self.model_name = model_name
        import torch
        from transformers import pipeline

        if self.model_loaded is None:
            print("LLM model loading from data section")
            
            self.model_loaded = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16,offload_folder="offload")
            #pipeline("text-generation", model="/packages/huggingface/cache/models--HuggingFaceH4--starchat-beta/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f/", torch_dtype=torch.bfloat16, device_map="cpu")
            self.tokenizer_loaded = AutoTokenizer.from_pretrained(model_name)
            
    def request_processing_fn(self, request) :
        """
        Processes Request Object -> List[{"question": "","context":""},{"question": "","context":""},...]

        :return: (n_inputs, payload's)

        Warnings:
        1. Do not reshape your final output for single sample here, do it in prediction.
           Else payloads will be invalidated for extraction at raw and extraction level.
        """
        final_payload = []
        raw_payload = request.json["payload"]
        return (len(raw_payload), raw_payload) 
    
    def pre_processing_fn(self,payload):
        return payload

    def get_user_question_for_open_chat(question,context):
        return """Now Based on this following context: \n """+ str(context) + "\n return a response for this question:\n" + question + "\n"
   
    
    def compose_prompt_open_chat(question,base_prompt,context):
        return f"""<s>[IDENTITY]
                {base_prompt}
                [/IDENTITY]</s>
                [INST]
                {get_user_question_for_open_chat(question,context)}
                [/INST]
                """
    def gereneate_output_openchat(prompt):
        #prompt = json.dumps(prompt)
        model_input = self.tokenizer_loaded(prompt, return_tensors="pt")#.to("cuda")
        #model.eval()
        response = self.tokenizer_loaded.decode(self.model_loaded.generate(**model_input, max_length=1024*6)[0], skip_special_tokens=True)
        return response
    
    def prediction_fn(self,
                      model: Any,
                      input_query,
                      base_prompt
                       ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
        question = input_query["question"]
        context = input_query["context"]
        promtpt = self.compose_prompt_open_chat(question=question,base_prompt=base_prompt,context=context)
        print("promting has started")
        outputs = self.gereneate_output_openchat(prompt)
        preds = outputs.split("[SOL]")[-1].split("[/SOL]")[0]
        print("prediction is \n: ",preds)
        return preds

In [5]:
context = context = {
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
                         "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
                        {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
                        {"ENTITY": "contribution_to_growth",
                         "other names": ["contribution to growth", "growth", "grown"]},
                        {"ENTITY": "kda_transactional",
                         "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
                        {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
                        {"ENTITY": "correlation",
                         "other names": ["associate", "associated", "association", "associations", "correlate",
                                         "correlated",
                                         "correlation", "correlations", "relate", "related", "relation", "relations",
                                         "relationship",
                                         "relationships"]}
                        ],
    "DATE VARIABLE": [
        {"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
}

In [7]:
score_ = ScoreTemplateExample(model_name="openchat/openchat_3.5")
import requests
req = requests.Request()
req.json = []

DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /openchat/openchat_3.5/resolve/main/config.json HTTP/1.1" 200 0


LLM model loading from data section


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /openchat/openchat_3.5/resolve/main/generation_config.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /openchat/openchat_3.5/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
score_.score(None, req, dry_run=True)

KeyError: 'payload'

In [24]:
score_ = ScoreTemplateExample()
import requests
req = requests.Request()

req.json = {"payload":"How do I build neural network in Python?"}

LLM model loading from data section


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of GPTBigCodeForCausalLM were not initialized from the model checkpoint at /packages/huggingface/cache/models--HuggingFaceH4--starchat-beta/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f/ and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
t3 = time.time()
model_predictions = score_.score(None, req, dry_run=True)
t4 = time.time()
t4-t3

,VALIDATION,COMPONENT,PASSED,SKIPPED
0,"Return Type Must Be Tuple (n_input, payloads)",request_processing_fn,True,False
1,"Tuple Must Be of length Two (n_input, payloads)",request_processing_fn,True,False
2,"* if n_input > 1 payload type must be List (n_input, [np.ndarray, tf.Tensor, etc])",request_processing_fn,True,False


Setting `pad_token_id` to `eos_token_id`:49155 for open-end generation.


Fields Marked Asterisk (*) Can Be Validated On Proper Input 

promting has started
prediction is 
:  
Building a neural network in Python can be done using the popular library TensorFlow. Here's a basic example of how to create a neural network in Python using TensorFlow:

import tensorflow as tf

# Define the layers of the neural network
input_layer = tf.keras.layers.Input(shape=(2,))
hidden_layer = tf.keras.layers.Dense(3, activation='relu')(input_layer)
output_layer = tf.keras.layers.Dense(1)(hidden_layer)

# Define the model
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Generate some sample data
x_train = [[0, 0], [1, 1], [2, 2], [3, 3]]
y_train = [0, 1, 4, 9]

# Train the model
model.fit(x_train, y_train, epochs=500, verbose=0)

# Evaluate the model
model.evaluate(x_train


159.01578283309937

In [26]:
print(model_predictions)

[<<ScoreResponse>>]


In [27]:
model="/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f"

In [28]:
register_model(model,
               ScoreTemplateExample,
               "Starchat_Beta_5worker_test",
               "Starchat_Beta_for_code_Generation",
               MLModelFlavours.pytorch,
               init_script="pip install SentencePiece \\n pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu"
              )

INFO:root:PASS:: Mandatory Validation :: dict_keys(['name', 'description', 'flavour', 'scoring_func'])
INFO:root:PASS:: AlphaNumeric Validation :: dict_keys(['name'])
INFO:root:PASS:: Validation/IfPresentTypeCheck :: dict_keys(['schema', 'metadata_info', 'tags'])
INFO:root:PASS:: Validation/IfPresentSubfieldMustExist :: dict_keys(['kyd'])
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "POST /registry/api/v1/ml-model/register HTTP/1.1" 200 10735
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "GET /registry/api/v1/ml-model/746254b5-39f1-4c3e-9520-04ed064c6a28 HTTP/1.1" 200 10728
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "GET /registry/api/v1/ml-model/746254b5-39f1-4c3e-9520-04ed064c6a28 HTTP/1.